# Dataset Creation
This notebook will be used to download and preprocess as much data as possible for the Forecasting notebook.


## Data requirements

- ideally in day or shorter intervals
- acceptable up to a month-long interval (but this will most likely eliminate all but linear models)


## Obvious dependencies

### G
- polish macroeconomic situation: interest rates, PLN exchange rates, GDP, etc.

### OP
- polish macroeconomic situation

### AP
- polish stock market: mostly WIG20
- polish macroeconomic situation

### ORR
- US, EUR, JAP macroeconomic situation

### ARR
- USD exchange rates (14 currencies -> might be able to reverse-engineer the fund structure easily)
- blue chip stocks: US(55%), JAP(8%), GB(6%)

### ORW
- USD exchange rates (60 countries => no reverse engineering)
- global macroeconomic situation, esp. emerging economies:
  Maxico(6%), China(5%), Indonesia(5%), Russia(4%), Turkey(4%)

### ARW
- USD exchange rates (?? countries => likely no reverse engineering)
- global macroeconomic situation
- stock markets: Hong-Konk(20%), S.Korea(15%), Taiwan(11%), India(8%), Brazil(7%), ..., Poland only 1.5%

In [1]:
import os
from time import sleep

import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

# Stock index data

In [2]:
stooq_page_base_url = 'https://stooq.pl/'
stock_index_url = f'{stooq_page_base_url}t/?i=510'
csv_download_dir = 'input/indices/'
csv_all_indices_path = 'input/all_indices_close.csv'

### Get list of the most importand stock indices from stooq.pl

In [3]:
response = requests.get(stock_index_url)
assert(response.status_code == 200)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table = soup.find(id='fth1')
tbody = table.find('tbody')
index_page_query_strings = [
    tr.find("a")["href"][2:]
    for tr in tbody.children if "r_" in tr.get('id', '')
]
len(index_page_query_strings), index_page_query_strings[:5]  # sanity check

(59, ['?s=^aor', '?s=^hsi', '?s=^jci', '?s=^klci', '?s=^kospi'])

### Download raw daily index data

In [5]:
if not os.path.exists(csv_download_dir):
    os.makedirs(csv_download_dir)

In [6]:
for ipqs in tqdm(index_page_query_strings):
    csv_url = f'https://stooq.pl/q/d/l/{ipqs}&i=d'
    stock_index_name = ipqs[3:]
    response = requests.get(csv_url)
    assert(response.status_code == 200)
    filename = os.path.join(csv_download_dir, f'{stock_index_name}.csv')
    with open(filename, 'w') as file:
        file.write(response.text)
    sleep(1)  # don't overload their server

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 59/59 [04:19<00:00,  5.16s/it]


### Convert data to a more friendly format

In [7]:
csv_names = os.listdir(csv_download_dir)
downloaded_csv_paths = [os.path.join(csv_download_dir, csv_name) for csv_name in csv_names]

In [8]:
daily_close_csvs = []
for csv_path, csv_name in tqdm(zip(downloaded_csv_paths, csv_names)):
    try:
        # keep only dates from 2000 to 2018 in the csv file
        index_name = csv_name[:4]
        df = pd.read_csv(csv_path, index_col='Data')
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        df = df.loc['2000-01-01':'2018-12-31']
        df.to_csv(csv_path, index=True)  # overwrite existing csv
        # add daily close values for the index to daily_close_csvs
        df_close = df[['Zamkniecie']].rename(columns={'Zamkniecie': index_name})
        daily_close_csvs.append(df_close)
    except Exception as e:
        print(csv_path)
        print(e)

59it [00:12,  3.95it/s]


In [9]:
len(daily_close_csvs)

59

In [10]:
all_indices_daily_close = pd.concat(daily_close_csvs, axis='columns')

In [11]:
all_indices_daily_close.head()

,a5.c,wig2,^aex,^aor,^ath,^bel,^bet,^bux,^bvp,^cac,...,^snx,^sof,^spx,^sti,^tdx,^tsx,^tws,^ukx,^ux.,^xu1
Data,,,,,,,,,,,,,,,,,,,,,
2000-01-03,1204.88,1852.9,675.44,NaN,5794.85,3311.05,NaN,NaN,16930.0,5917.37,...,5375.11,NaN,1455.22,2582.94,5151.36,NaN,NaN,NaN,94.79,NaN
2000-01-04,1194.41,1796.6,642.25,3124.1,5588.65,3188.09,NaN,8709.17,15851.0,5672.02,...,5491.01,NaN,1399.42,2530.15,4977.74,8202.60,8756.55,6665.9,94.79,17512.19
2000-01-05,1192.89,1777.0,632.31,3050.9,5369.38,3057.53,NaN,8464.34,16245.0,5479.70,...,5357.00,NaN,1402.11,2391.03,4810.34,8119.42,8849.87,6535.9,94.79,16932.00
2000-01-06,NaN,1832.1,624.21,3030.1,NaN,3061.09,NaN,8483.29,16107.0,5450.11,...,5421.53,NaN,1403.45,2359.21,4789.80,8114.24,8922.03,6447.2,94.79,16200.02
2000-01-07,1223.61,1933.2,644.86,3044.5,5410.82,3138.55,NaN,8694.04,16309.0,5539.61,...,5414.48,NaN,1441.47,2406.04,4984.45,8429.42,8845.47,6504.8,94.79,15837.36


In [12]:
pct_missing = all_indices_daily_close.isna().sum() * 100 / len(all_indices_daily_close)
pct_missing.sort_values(ascending=False).head(10)

^sof    15.574263
^bet     9.420726
^nz5     9.340549
^shb     8.538785
^shc     7.777110
^jci     7.456404
^sax     6.895169
^set     6.794949
^mrv     6.714772
^klc     6.514332
dtype: float64

For all indices but one, >90% of days are available. This is quite good and we can work on such data.

In [13]:
all_indices_daily_close.to_csv(csv_all_indices_path, index=True)

# Commodity prices

Code below is copied from above, in case there is a need for tweaking / changing something it might be useful to move this functionality into a small, separate package.

In [14]:
stooq_page_base_url = 'https://stooq.pl/'
stock_index_url = f'{stooq_page_base_url}t/?i=512'
csv_download_dir = 'input/commodities/'
csv_all_indices_path = 'input/all_commodities_close.csv'

### Get list of the most importand commodity prices from stooq.pl

In [15]:
response = requests.get(stock_index_url)
assert(response.status_code == 200)
soup = BeautifulSoup(response.text, 'html.parser')

In [16]:
table = soup.find(id='fth1')
tbody = table.find('tbody')
index_page_query_strings = [
    tr.find("a")["href"][2:]
    for tr in tbody.children if "r_" in tr.get('id', '')
]
len(index_page_query_strings), index_page_query_strings[:5]  # sanity check

(41, ['?s=cb.f', '?s=ck.f', '?s=cl.f', '?s=fi.f', '?s=ho.f'])

### Download raw daily index data

In [17]:
if not os.path.exists(csv_download_dir):
    os.makedirs(csv_download_dir)

In [18]:
for ipqs in tqdm(index_page_query_strings):
    csv_url = f'https://stooq.pl/q/d/l/{ipqs}&i=d'
    stock_index_name = ipqs[3:]
    response = requests.get(csv_url)
    assert(response.status_code == 200)
    filename = os.path.join(csv_download_dir, f'{stock_index_name}.csv')
    with open(filename, 'w') as file:
        file.write(response.text)
    sleep(1)  # don't overload their server

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [02:33<00:00,  4.50s/it]


### Convert data to a more friendly format

In [19]:
csv_names = os.listdir(csv_download_dir)
downloaded_csv_paths = [os.path.join(csv_download_dir, csv_name) for csv_name in csv_names]

In [20]:
daily_close_csvs = []
for csv_path, csv_name in tqdm(zip(downloaded_csv_paths, csv_names)):
    try:
        # keep only dates from 2000 to 2018 in the csv file
        index_name = csv_name[:4]
        df = pd.read_csv(csv_path, index_col='Data')
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        df = df.loc['2000-01-01':'2018-12-31']
        df.to_csv(csv_path, index=True)  # overwrite existing csv
        # add daily close values for the index to daily_close_csvs
        df_close = df[['Zamkniecie']].rename(columns={'Zamkniecie': index_name})
        daily_close_csvs.append(df_close)
    except Exception as e:
        print(csv_path)
        print(e)

41it [00:08,  3.87it/s]


In [21]:
len(daily_close_csvs)

41

In [22]:
all_indices_daily_close = pd.concat(daily_close_csvs, axis='columns')

In [23]:
all_indices_daily_close.head()

,ca.f,cb.f,cc.f,ck.f,cl.f,ct.f,dl.f,fi.f,gc.f,gf.f,...,ux.f,xr.f,zc.f,zk.f,zl.f,zm.f,zo.f,zr.f,zs.f,zw.f
Data,,,,,,,,,,,,,,,,,,,,,
2000-01-03,NaN,NaN,830.0,NaN,NaN,51.07,10.26,NaN,NaN,85.90,...,NaN,NaN,200.75,NaN,15.70,146.7,108.25,5.48,456.50,247.50
2000-01-04,561.0,24.39,836.0,NaN,25.55,50.73,10.27,NaN,283.7,85.70,...,NaN,NaN,203.00,NaN,15.82,148.6,109.00,5.50,464.25,247.25
2000-01-05,560.0,23.73,831.0,NaN,24.91,51.55,10.20,NaN,NaN,85.70,...,NaN,NaN,NaN,NaN,15.96,151.8,109.25,5.51,469.25,249.75
2000-01-06,556.0,23.62,841.0,NaN,24.78,52.08,10.05,NaN,282.4,86.38,...,NaN,NaN,203.75,NaN,16.05,150.4,109.00,5.53,468.00,248.50
2000-01-07,564.0,23.09,853.0,NaN,24.22,53.96,10.02,NaN,282.9,86.88,...,NaN,NaN,207.00,NaN,16.19,150.7,109.50,5.53,471.50,251.75


In [24]:
pct_missing = all_indices_daily_close.isna().sum() * 100 / len(all_indices_daily_close)
pct_missing.sort_values(ascending=False).head(10)

fi.f    79.828851
ux.f    67.135289
tr.f    64.221679
rm.f    46.332518
ck.f    43.765281
lu.f    36.246944
zk.f    30.052975
xr.f    10.044825
rs.f     4.339853
dl.f     4.156479
dtype: float64

Not as reliable as stocks, but still should be fine for forecasting.

In [25]:
all_indices_daily_close.to_csv(csv_all_indices_path, index=True)